In [25]:
import filefit
import kex_data
import e7tools
import importlib
importlib.reload(kex_data)

<module 'kex_data' from 'C:\\Users\\petct\\Desktop\\Hugo\\Code\\PythonMEX\\kex_data.py'>

In [7]:
# plan
# how does it work? 
#       what is the bare minimum of input?
#       does it create new files? (start with not force) (it seems to be changing the norm file)
# what happens when changing the emission input to normalized emission?

In [34]:
emissions = kex_data.get_sino_mainhdrs()
print("emissions\n", emissions)
norm_hdr = kex_data.NORM_PATH
print("norm\n", norm_hdr)
dicom_paths = kex_data.paths(both = True)[-1]
v_paths = [path[:-len("-DICOM")] for path in dicom_paths]
v_mhdrs = [kex_data.get_v_mhdr(path) for path in v_paths]
print("images\n", v_mhdrs)

emissions
 ['C:/Users/petct/Desktop/CylinderPET-Converted/CylinderPET-LM-00/CylinderPET-LM-00-sino.mhdr', 'C:/Users/petct/Desktop/CylinderPET-Converted/CylinderPET-LM-01/CylinderPET-LM-01-sino.mhdr', 'C:/Users/petct/Desktop/KanylPET-Converted/KanylPET-LM-00/KanylPET-LM-00-sino.mhdr', 'C:/Users/petct/Desktop/KanylPET-Converted/KanylPET-LM-01/KanylPET-LM-01-sino.mhdr']
norm
 C:/Users/petct/Desktop/KanylPET-Converted/KanylPET-norm.n.hdr
images
 ['C:/Users/petct/Desktop/CylinderPET-Converted/CylinderPET-LM-00/CylinderPET-LM-00-OPTOF.mhdr', 'C:/Users/petct/Desktop/CylinderPET-Converted/CylinderPET-LM-01/CylinderPET-LM-01-OPTOF.mhdr', 'C:/Users/petct/Desktop/KanylPET-Converted/KanylPET-LM-00/KanylPET-LM-00-OPTOF.mhdr', 'C:/Users/petct/Desktop/KanylPET-Converted/KanylPET-LM-01/KanylPET-LM-01-OPTOF.mhdr']


In [35]:
workplace= r"C:/Users/petct/Desktop/Hugo/Code/PythonMEX/e7_recon"
filling = [norm_hdr, norm_hdr.replace(".hdr", "")]
filefit.prep_folder(workplace, filling)
norm_hdr = filefit.refolder_path(workplace, norm_hdr)

In [21]:
e7tools.run_tool('e7_norm -h')

stdout
 b'
                                                   (c) 2007-2015 Siemens Healthcare MI

e7_norm - compute normalization components

Usage:

 e7_norm (--act activity -i image --itime injectionTime -e emission[,r,t,dim]
          -u u-map -s scatter[,r,t,dim] -n norm[,r,t] --ccf ccf
          --rad phantom_radius --tof --gpu 0|1,2,... --vol phantom_vol
          [--prj none|ifore|issrb|fwdpj] [--fsum] [--force] [-d path]
          [-l a[,path]] [--resrv ip,port,port])|-v|-h|-?

  --act   injected activity in the calibration phantom, unit (Bq). This value 
          is used in ECF calculation.
           Example: --act 60000.0
  -i      name of image input file. Supported formats are ECAT7, Interfile and
          flat (IEEE-float, little endian). In case of a flat-file, the geometry
          of the gantry needs to be specified by the "--model" switch. The width
          and height of the image are assumed to be the same as the projection
          width of the gantry if this

0

In [137]:
params = ['act', 'i', 'itime', 'e', 'u', 's', 'n', 'ccf', 'rad', 'tof', 'gpu', 'vol', 'prj', 'fsum', 'force', 'd', 'l']
num_dashes = [2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1]
dashes = ['-'*n for n in num_dashes]
print(dashes)
print(dict(zip(params, num_dashes)))
param_values = [None]*len(params)
param_key_values = dict(zip(params, param_values))
print(param_key_values)

['--', '-', '--', '-', '-', '-', '-', '--', '--', '--', '--', '--', '--', '--', '--', '-', '-']
{'act': 2, 'i': 1, 'itime': 2, 'e': 1, 'u': 1, 's': 1, 'n': 1, 'ccf': 2, 'rad': 2, 'tof': 2, 'gpu': 2, 'vol': 2, 'prj': 2, 'fsum': 2, 'force': 2, 'd': 1, 'l': 1}
{'act': None, 'i': None, 'itime': None, 'e': None, 'u': None, 's': None, 'n': None, 'ccf': None, 'rad': None, 'tof': None, 'gpu': None, 'vol': None, 'prj': None, 'fsum': None, 'force': None, 'd': None, 'l': None}


In [44]:
def build_param_str(params, num_dashes):
    param_str = ""
    for (key, value), n in zip(params.items(), num_dashes):
        if value is not None:
            param_str = param_str + " {}{} {} ".format('-'*n, key, value) 
    return param_str

In [150]:
param_map = dict(zip(params, param_values))
param_str = build_param_str(param_map, num_dashes)
print("param_str:", param_str)
param_map['e'] == emissions[0]
param_map['n'] = norm_hdr
param_map['i'] = v_mhdrs[2]
#param_map['i'] = dicom_paths[2] a folder, but e7_norm requires  files
param_map['act'] = 1000
param_map['itime'] = 1500000000000
param_map['d'] = workplace
param_str = build_param_str(param_map, num_dashes)
param_map['force'] = ""
param_map['tof'] = ""
param_map['rad'] = 100
param_map['vol'] = 7000
param_map['prj'] = 'ifore'
param_map['fsum'] = ""
param_map['ccf'] = 10
param_str = build_param_str(param_map, num_dashes)
print("param_str:", param_str)

param_str: 
param_str:  --act 1000  -i C:/Users/petct/Desktop/KanylPET-Converted/KanylPET-LM-00/KanylPET-LM-00-OPTOF.mhdr  --itime 1500000000000  -n C:/Users/petct/Desktop/Hugo/Code/PythonMEX/e7_recon/KanylPET-norm.n.hdr  --ccf 10  --rad 100  --tof   --vol 7000  --prj ifore  --fsum   --force   -d C:/Users/petct/Desktop/Hugo/Code/PythonMEX/e7_recon 


In [151]:
e7tools.run_tool("e7_norm " + param_str)

stdout
 b''
stderr
 b'I 06/03/2020 09:15:32.886 (UTC+01:00) Intel(R) Core(TM) i7-7820HQ CPU @ 2.90GHz (1x)    memory: 32629 MByte
I 06/03/2020 09:15:32.886 (UTC+01:00) Number of NUMA nodes: 1
I 06/03/2020 09:15:32.886 (UTC+01:00) Number of physical processor packages: 1
I 06/03/2020 09:15:32.886 (UTC+01:00) Number of processor cores: 4
I 06/03/2020 09:15:32.886 (UTC+01:00) Number of logical processors: 8
I 06/03/2020 09:15:32.886 (UTC+01:00) Number of processor L1/L2/L3 caches: 8/4/1 
I 06/03/2020 09:15:32.886 (UTC+01:00) Hyper-Threadingin BIOS Enabled.
I 06/03/2020 09:15:32.886 (UTC+01:00)  supported CPU features: MMX,SSE,SSE2,SSE3,SSE4.1,SSE4.2,AVX
I 06/03/2020 09:15:33.399 (UTC+01:00) GPU Driver Version: 24.20.100.6286
I 06/03/2020 09:15:33.400 (UTC+01:00) GPU Driver supported CUDA Version: 9010
I 06/03/2020 09:15:33.400 (UTC+01:00) CUDA Run-time Version Used: 5000
I 06/03/2020 09:15:33.400 (UTC+01:00) Quadro M620 1017 MHz (Compute Capability 5.0)
I 06/03/2020 09:15:33.400 (UTC+01:0

0

In [133]:
# The calculated ECF is 21.1398
# 23.9099
# 21.1398

In [134]:
n_path = norm_hdr.replace(".hdr", "")
import numpy as np
with open(n_path, 'rb') as f:
    norm_components = np.fromfile(n_path, dtype=np.float32)

In [135]:
#norm_components2 = norm_components

In [136]:
diff = norm_components2 - norm_components
print(np.sum(diff))

0.0
